# Állomások lokációi és a legközelebbi meteorológia mérőállomás

In [1]:
import pandas as pd

In [2]:
from config import data_root,weather_folder,gtfs_folder,gtfs_stops_file,passanger_info_folder,years_to_process
'''data_root="data/"
weather_folder=data_root+"odp/"
gtfs_folder=data_root+"gtfsMavMenetrend/"
gtfs_stops_file="stops.txt"
'''
def get_location_data():
    gtfs_stops=pd.read_csv(gtfs_folder+gtfs_stops_file)
    locs=[]
    passanger_info_folders=[passanger_info_folder+str(y)+"/" for y in years_to_process]
    for l in passanger_info_folders:
        passanger_info_locations_file=l+"t_szolg_helyek.txt"
        places_=pd.read_csv(passanger_info_locations_file,sep=',',encoding='iso-8859-2')
        locs.append(places_)
    places=pd.concat(locs,axis=0)
    for l in locs:
        del l
    #TODO check multiple appearence
    places=places.groupby('TELJES_NEV').agg("last")
    places_with_gtfs=places.merge(gtfs_stops,how='left', left_on='POLGARI_NEV', right_on='stop_name')
    return places_with_gtfs


In [3]:
locations=get_location_data()

In [4]:
locations.head()

,OR_KOD,SZH_KOD,SZH_NEV,POLGARI_NEV,SZH_TIP_KOD,TEHERVONAT,UTASFORGALOM,UZEMVALT,HATARALLOMAS,UZEM_VVALL_VPE_ID,...,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,stop_timezone,wheelchair_boarding
0,55,5504135,Aba-Sárkeresztúr,Aba-Sárkeresztúr,1,1.0,1.0,0,0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,55,5507229,Abaliget,Abaliget,1,1.0,1.0,0,0,2.0,...,Abaliget,NaN,46.152778,18.073611,NaN,NaN,0.0,NaN,NaN,2.0
2,55,5515610,Abapuszta,Abapuszta,2,0.0,1.0,0,0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,55,5513326,Abaújkér,Abaújkér,2,0.0,1.0,0,0,2.0,...,Abaújkér,NaN,48.309444,21.198333,NaN,NaN,0.0,NaN,NaN,2.0
4,55,5513326,Abaújkér,Abaújkér,2,0.0,1.0,0,0,2.0,...,Abaújkér,NaN,48.309444,21.198333,NaN,NaN,0.0,NaN,NaN,2.0


In [5]:
from math import radians, sin, cos, sqrt, atan2

# Function to calculate distance
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371.0  # Earth radius in kilometers

    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance
'''
# Coordinates of two locations
lat1, lon1 = 52.2296756, 21.0122287  # Warsaw, Poland
lat2, lon2 = 41.8919300, 12.5113300  # Rome, Italy

# Calculate the distance
distance = calculate_distance(lat1, lon1, lat2, lon2)
print(f"The distance is {distance} kilometers.")
'''

'\n# Coordinates of two locations\nlat1, lon1 = 52.2296756, 21.0122287  # Warsaw, Poland\nlat2, lon2 = 41.8919300, 12.5113300  # Rome, Italy\n\n# Calculate the distance\ndistance = calculate_distance(lat1, lon1, lat2, lon2)\nprint(f"The distance is {distance} kilometers.")\n'

In [6]:
weather_meta_file_name=weather_folder+"weather_meta_avg.csv"
met_stat_locations=pd.read_csv(weather_meta_file_name,sep=',',encoding='iso-8859-2')
met_stat_locations

,Loc,Lat,Lon
0,Budapest,47.507300,19.031400
1,Debrecen,47.531438,21.625387
2,Keszthely,46.742617,17.227400
3,Miskolc,48.123882,20.837100
4,Nyiregyhaza,47.969911,21.767344
5,Pecs,46.087188,18.277832
6,Sopron,47.687800,16.603233
7,Szeged,46.251943,20.114371
8,Szombathely,47.249480,16.623060
9,Turkeve,47.105900,20.734733


In [7]:
def get_min_dist_loc(loc_df,lat,lon):
    loc_df['d']=loc_df.apply(lambda x: calculate_distance(lat,lon,x['Lat'],x['Lon']),axis=1)
    return loc_df.loc[loc_df['d'].idxmin()]['Loc']
def get_Locations_coordinates(locations_df,met_locations):
    coord_dict=locations_df[['SZH_KOD','stop_lat','stop_lon']].loc[locations_df['stop_lat'].notna()]
    coord_dict['SZH_KOD']=coord_dict['SZH_KOD'].apply(lambda x: str(x)[2:]).astype(int)
    coord_dict['Legközelebbi állomás']=coord_dict.apply(lambda x: get_min_dist_loc(met_locations,x['stop_lat'],x['stop_lon']),axis=1)
    return coord_dict
coord_dict=get_Locations_coordinates(locations,met_stat_locations)

In [8]:
coord_dict.head()

,SZH_KOD,stop_lat,stop_lon,Legközelebbi állomás
1,7229,46.152778,18.073611,Pecs
3,13326,48.309444,21.198333,Miskolc
4,13326,48.309444,21.198333,Miskolc
5,13300,48.286944,21.181667,Miskolc
6,13318,48.268333,21.179722,Miskolc


In [9]:
'''
event_file_name="vonat_esemenyek_"

events=pd.read_csv(passanger_info_folder+event_file_name+str(2)+'.txt',sep=',',encoding='iso-8859-2')
events.head()
'''

'\nevent_file_name="vonat_esemenyek_"\n\nevents=pd.read_csv(passanger_info_folder+event_file_name+str(2)+\'.txt\',sep=\',\',encoding=\'iso-8859-2\')\nevents.head()\n'

In [10]:
#a=events['IDO'].unique()

In [11]:
#days=pd.to_datetime(events['KOZLEKEDESI_NAP'],format='%Y.%m.%d',errors='coerce')

In [12]:
'''
min_date=days.min()
max_date=days.max()
print(min_date,' - ',max_date)
'''

"\nmin_date=days.min()\nmax_date=days.max()\nprint(min_date,' - ',max_date)\n"

Mit használunk a késésnél előrejelzésnél?
- Szakasz: 
- előző késés
- előző vonat áthaladás
- előző vonat késés 

- szakaszhoz hozzáadjuk pálya állapotot

potenciálisan :
- átlagsebesség a szakaszon



In [13]:
weather_file_name='weather.csv'
import datetime

weather_df=pd.read_csv(weather_folder+weather_file_name,sep=',',encoding='iso-8859-2',parse_dates=['Time'],date_format='%Y-%m-%d')
#weather_df=weather_df.loc[weather_df['Time'].between(min_date,max_date)]


In [14]:
weather_df.head()

,Loc,Time,tx,t,tn,r
0,Budapest,1901-01-01,-0.4,-5.7,-9.2,1.9
1,Budapest,1901-01-02,-6.6,-9.3,-11.3,0.0
2,Budapest,1901-01-03,-6.6,-9.1,-10.8,0.8
3,Budapest,1901-01-04,-9.8,-11.0,-12.4,0.2
4,Budapest,1901-01-05,-9.0,-11.1,-15.5,0.0


In [20]:
from config import event_file_name
import numpy as np


date_format='%Y.%m.%d %H:%M:%S'
drop_cols=['IND_CEL_JELZO','HATARESEM_JELZO','KOFE_IDO','KOFE_MODDAT']


# TODO lehet évenként kezelni
def process_events(events):
    
    events=events.drop(columns=drop_cols)
    events['IDO']=pd.to_datetime(events['IDO'],format=date_format,errors='coerce')
    events['TENY_IDO']=pd.to_datetime(events['TENY_IDO'],format=date_format,errors='coerce')
    events['OSSZ_KESES']=events['TENY_IDO']-events['IDO']

    events['SZH_KOD']=pd.to_numeric(events.apply(lambda x:'%s%s' % (x['ORSZAG_KOD'],x['ALLOMAS']),axis=1))


    events[['ELOZO_KESES','ELOZO_ALLOMAS','ELOZO_ESEMENY_IDO','ELOZO_SZH_KOD']]=events.groupby(['VONATSZAM','KOZLEKEDESI_NAP'])[['OSSZ_KESES','ALLOMAS','IDO','SZH_KOD']].shift()
    events['KESES']=events['OSSZ_KESES']-events['ELOZO_KESES'] 

    events['TERV_IDOTARTAM (m)']= (events['IDO']-events['ELOZO_ESEMENY_IDO']).dt.seconds/60
    events['RELATIV_KESES']=  (events['KESES']/np.timedelta64(1, 'm'))/events['TERV_IDOTARTAM (m)']

    events[['ELOZO_TERV_IDOTARTAM (m)','ELOZO_RELATIV_KESES']]=events.groupby(['VONATSZAM','KOZLEKEDESI_NAP'])[['TERV_IDOTARTAM (m)','RELATIV_KESES']].shift()


   
    events[['ELOZO_ALLOMAS','ELOZO_SZH_KOD']]=events[['ELOZO_ALLOMAS','ELOZO_SZH_KOD']].fillna(0)
    events['ELOZO_ALLOMAS'] = events['ELOZO_ALLOMAS'].astype(int)
    events['ELOZO_SZH_KOD'] = events['ELOZO_SZH_KOD'].astype(int)


    # TODO duplicate?
    events['KOZLEKEDESI_NAP']=pd.to_datetime(events['KOZLEKEDESI_NAP'],format="%Y.%m.%d",errors='coerce')
    #events['KOZLEKEDESI_NAP_DT']=events['IDO'].dt.date
    events['MENETREND_IDO']=events['IDO'].dt.time
    #events['prev_Delay']=events.groupby(['VONATSZAM','KOZLEKEDESI_NAP'])['Delay'].shift()
    min_date=events['KOZLEKEDESI_NAP'].min()
    max_date=events['KOZLEKEDESI_NAP'].max()
    print(min_date,' - ',max_date)
    return events,min_date,max_date

def add_weather_data(coord_dict,weather_df,events):
    #events = e.copy()
    events['IndulóÁllomás']=events.groupby(['VONATSZAM','KOZLEKEDESI_NAP']).first().reset_index()['ALLOMAS'].astype(int)
    events=events.merge(coord_dict, how='left', left_on='IndulóÁllomás',right_on='SZH_KOD')
    events = events.merge(weather_df, how='left', left_on=['Legközelebbi állomás', 'KOZLEKEDESI_NAP'],
                              right_on=['Loc', 'Time'])
    return events

def get_events_data(weather_df,coord_dict,year=2022,line=2 ):
    events_=[]
    l= passanger_info_folder + str(year) + "/"
    e_=pd.read_csv(l+event_file_name+str(line)+'.txt',sep=',',encoding='iso-8859-2')
    events_.append(e_)
    
    e=pd.concat(events_,axis=0)
    del events_
    e,min_date,max_date=process_events(e)
    weather_df=weather_df.loc[weather_df['Time'].between(min_date,max_date)]

    e=add_weather_data(coord_dict,weather_df,e)
    junk_from_joins=['Loc','SZH_KOD_x','SZH_KOD_y','stop_lat','stop_lon','Time','Legközelebbi állomás','IndulóÁllomás']
    
    
    e.drop(columns=junk_from_joins,inplace=True)
    return e

events_2=get_events_data(weather_df,coord_dict)  



C:\Users\ELTE\AppData\Local\Temp\ipykernel_7280\2781791593.py:56: DtypeWarning: Columns (16,17,28) have mixed types. Specify dtype option on import or set low_memory=False.
  e_=pd.read_csv(l+event_file_name+str(line)+'.txt',sep=',',encoding='iso-8859-2')


2022-01-01 00:00:00  -  2023-12-31 00:00:00


In [21]:
import numpy as np
#elegánsabb lenne VONATSZAM és VONAL nélkül, de ahhot több adat kell majd..
# ELOZO_ESEMENY_IDO a line status joinhoz -terv
#TODO  LINE egy közelítés az állapot leírására, ezen majd lehet dolgozni azért...
#TODO ELOZO_KESES elkodolhat maradek információkat ezen is lehet majd dolgozni, bár rendenkezésre állhat
#TODO VONATSZAM szintén menetrendi, prioritás információk, de túl sok lehetőség,és ellentmond a logikánknak inkább érdemes dobni
#érdekes lehet : MOZDONYAZONOSITO, UIC, VONTATO_TIPUS
to_keep=['VONAL','VONATSZAM','VONTATO_TIPUS','IDO','ELOZO_ESEMENY_IDO','ALLOMAS','ELOZO_ALLOMAS','MENETREND_IDO','OSSZ_KESES','ELOZO_KESES','KESES','RELATIV_KESES','TERV_IDOTARTAM (m)','tx','t','tn','r']
def get_events_pred_data(processed, line=2):
    #ezek azok amik kellenek most, és később is
    processed=processed[to_keep]
    processed['LINE']=line
    processed['ID']=  processed['ELOZO_ALLOMAS'].astype(str)+"-"+processed['ALLOMAS'].astype(str)

    processed.drop(columns=['ELOZO_ALLOMAS','ALLOMAS','IDO'],inplace=True)
    return processed
d2=get_events_pred_data(events_2,2)
d2.to_pickle(data_root+'test_events_data.pkl')


C:\Users\ELTE\AppData\Local\Temp\ipykernel_7280\759602514.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processed['LINE']=line
C:\Users\ELTE\AppData\Local\Temp\ipykernel_7280\759602514.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processed['ID']=  processed['ELOZO_ALLOMAS'].astype(str)+"-"+processed['ALLOMAS'].astype(str)
C:\Users\ELTE\AppData\Local\Temp\ipykernel_7280\759602514.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats 

# Vonatfej - TODO
nem tudom mire lesz még jó..

In [ ]:
#TODO 
def get_train_head(line=2):
    heads=[]
    for l in  [passanger_info_folder+str(y)+"/" for y in years_to_process]:
        fn = l + f'vonat_fej_{str(line)}.txt'
        h = pd.read_csv(fn,sep=',',encoding='iso-8859-2')
        heads.append(h)
    train_head=pd.concat(heads,axis=0)    
    train_head['ERK_IDO_MN']=pd.to_datetime(train_head['ERK_IDO'],format='%Y.%m.%d %H:%M:%S').dt.time
    train_head['IND_IDO_MN']=pd.to_datetime(train_head['IND_IDO'],format='%Y.%m.%d %H:%M:%S').dt.time
    return train_head

head_2=get_train_head()

In [ ]:
head_2.head()

In [ ]:
def get_different_heads(train_head,train_id):
    if train_id:
        train_head=train_head.loc[train_head['VONATSZAM']==train_id]
    train_head['KOZLEKEDESI_NAP_DT']=pd.to_datetime(train_head['IND_IDO'],format='%Y.%m.%d %H:%M:%S').dt.date
    separate_trains_in_schedule=train_head[['VONATSZAM','VNEM_KOD','IND_ALL_KOD','CEL_ALL_KOD','CEL_ESEMENY_SORSZAM','KOZLEKEDESI_NAP_DT']].drop_duplicates()
    route_start_ends= separate_trains_in_schedule.groupby(['VONATSZAM','IND_ALL_KOD','CEL_ALL_KOD','CEL_ESEMENY_SORSZAM'])['KOZLEKEDESI_NAP_DT'].agg(first="min",last="max",count="count")
    return route_start_ends
def get_different_schedules(head,events,train_id=None):
    train_event_data=events
    if train_id:
        train_event_data=train_event_data.loc[train_event_data['VONATSZAM']==train_id]
    route_start_ends = get_different_heads(head,train_id)
    schedule_variants=train_event_data.loc[train_event_data['KOZLEKEDESI_NAP_DT'].isin(route_start_ends['first'])]
    return schedule_variants

    
    
    

variants = get_different_schedules(head_2,events_2,train_id=2010)    

# Menetrend variációk Vizualizációja

In [ ]:
import datetime

variants = variants[['KOZLEKEDESI_NAP_DT', 'ELOZO_ALLOMAS', 'ELOZO_ESEMENY_IDO', 'ALLOMAS', 'MENETREND_IDO']].dropna()
variants['ELOZO_ESEMENY_IDO']=variants['ELOZO_ESEMENY_IDO'].dt.time
# todo astype to creation

variants.head()

In [ ]:

today_ = datetime.date.today()
variants['Start'] = variants.apply(
    lambda x: datetime.datetime.combine(today_, x['ELOZO_ESEMENY_IDO']), axis=1)
variants['Finish'] = variants.apply(
    lambda x: datetime.datetime.combine(today_, x['MENETREND_IDO']), axis=1)
variants['Task'] = variants['ELOZO_ALLOMAS'].astype(str) + '-' + \
                                     variants['ALLOMAS'].astype(str)



In [ ]:
import plotly.express as px

fig = px.timeline(variants, x_start="Start", x_end="Finish", y="KOZLEKEDESI_NAP_DT", color="Task")
fig.update_yaxes(autorange="reversed", dtick=1)

fig.show()

### Data Preprocessing 

# Lassújel és vágányzár

In [ ]:
import pandas as pd

slow_signal_tables = pd.read_excel('data/obstacles/lassújel_2_147_vonal_20220101-20231231_AI-hoz.xlsx',
                                   sheet_name=['ZVZ_KEIHEAD', 'ZVZ_KIZARASOK', 'ZVZ_VZTHEAD'])
closure_tables = pd.read_excel('data/obstacles/vágányzár_2_147_vonal_20220101-20231231_AI-hoz.xlsx    ',
                               sheet_name=['ZVZ_KIZARASOK', 'ZVZ_VZTHEAD'])


In [ ]:
ss_parts=slow_signal_tables['ZVZ_KIZARASOK'].copy()
ss_parts.drop(axis=0,index=0,inplace=True)


ss_parts['Kizárásszám']=ss_parts['Kizárásszám'].astype(str)


to_drop=['Kizárás tétel','Kizárás típusa','Kezdő SZH statisztikai szám','Záró SZH statisztikai szám','Lassúmenet kezdő szelvény','Lassúmenet záró szelvény','Változó munkahelyen kitűzött','Lassúmenet mértéke tehervonatra (km/h)','Kis teljesítményű dízel kitoló','Dízel kitoló szükséges','Nagy teljesítményű dízel kitoló', 'Állomási bizt.ber. kikapcsolás szükséges','Vonali bizt. ber. kikapcsolás szükséges',"INF vonal","INF vonal.1","HÜSZ vonal","HÜSZ vonal.1"]
#todo később jól jöhetnek: 
# Lassúmenet kezdő szelvény, Lassúmenet záró szelvény,Lassúmenet mértéke tehervonatra (km/h),Kis teljesítményű dízel kitoló (csupa 0),Dízel kitoló szükséges (nincs ilyen),Nagy teljesítményű dízel kitoló (nincs ilyen),Állomási bizt.ber. kikapcsolás szükséges(nincs ilyen)
# kérdéses: Változó munkahelyen kitűzött, Kizárás típusa (redundáns?)
#todo kezdő, záró szolgálati hely mindig egy minimális szakasz?
cols= ss_parts.columns
for d in to_drop:
    if d not in cols:
        print('Missing:',d)
    
ss_parts.drop(columns=to_drop,inplace=True)

#lassúmenet iránya egy adott sínpárra/vágányra vonatkozik, de sínpár is lehet
# Vágányzár VZ írja le melyik vágányról beszélünk.

# ha > akkor a jobb oldali lehet érdekesebb 
#ss_parts['S_ID']=ss_parts.apply(lambda x: transform_SZH_KOD(x['Kezdő szolgálati hely'],x['Záró szolgálati hely']) if ">" in x["Lassúmenet iránya"] else -1   ,axis=1)

# ha  < akkor a bal 
#ss_parts['S_ID_R']=ss_parts.apply(lambda x: transform_SZH_KOD(x['Záró szolgálati hely'],x['Kezdő szolgálati hely']) if "<" in x["Lassúmenet iránya"] else -1 ,axis=1)

# kellene olyasmi hogy Irány szerinti oldal, és ellentétes oldal lezárva ehhez érdemes lehet kettészedni az irányokat..
# todo: VZ van jobb/bal oldali közlekedés? most abból indulunk ki, hogy van valmilyen szinten azaz szakszmenti és ellentétes - J: I(rány) a nem fordítottnál, E(llentétes) a fordítottnál, B  E(llenttes) a nem fordítottnál és I a fordítottnál

#todo VZ elméletben lehet A is, gyakorlatban csak 1 és 4 van
ss_parts.loc[ss_parts['Záró szolgálati hely'].isna(),'Záró szolgálati hely']=ss_parts['Kezdő szolgálati hely']

ss_parts_fw = ss_parts[ss_parts['Lassúmenet iránya'].str.contains(">")]
ss_parts_bw = ss_parts[ss_parts['Lassúmenet iránya'].str.contains("<")]

def set_section_id(df):
    df['Kezdő szolgálati hely']=df['Kezdő szolgálati hely'].str.replace('M-55','')
    df['Záró szolgálati hely']=df['Záró szolgálati hely'].str.replace('M-55','')
    
    df['ID']=df['Kezdő szolgálati hely'].astype(str)+"-"+df['Záró szolgálati hely'].astype(str)

def encode_directions(dff,dfb):
    dff["I"]=dff.apply(lambda x: 1 if x['Vágány (VZ)'] in [1,3,4]  else 0,axis=1)
    dff["E"]=dff.apply(lambda x: 1 if x['Vágány (VZ)'] in [2,3,4]  else 0,axis=1)
    dff["ÁI"]=dff.apply(lambda x: 1 if x['Átmenő vágány típusa'] in ['A','Á','J']  else 0,axis=1)
    dff["ÁE"]=dff.apply(lambda x: 1 if x['Átmenő vágány típusa'] in ['A','Á','B']  else 0,axis=1)
    
    dfb["I"]=dfb.apply(lambda x: 1 if x['Vágány (VZ)'] in [2,3,4]  else 0,axis=1)
    dfb["E"]=dfb.apply(lambda x: 1 if x['Vágány (VZ)'] in [1,3,4]  else 0,axis=1)
    dfb["ÁI"]=dfb.apply(lambda x: 1 if x['Átmenő vágány típusa'] in ['A','Á','B']  else 0,axis=1)
    dfb["ÁE"]=dfb.apply(lambda x: 1 if x['Átmenő vágány típusa'] in ['A','Á','J']  else 0,axis=1)
    dfb[['Kezdő szolgálati hely','Záró szolgálati hely']]=dfb[['Záró szolgálati hely','Kezdő szolgálati hely']]
    return pd.concat([dff,dfb],axis=0)


ss_parts=encode_directions(ss_parts_fw,ss_parts_bw)
set_section_id(ss_parts)

ss_parts = ss_parts.drop(columns=['Kezdő szolgálati hely','Záró szolgálati hely','Lassúmenet iránya','Vágány (VZ)','Átmenő vágány típusa']) 

ss_parts.head()

In [ ]:
# csak lassújelnél
# helyettesítsük 0-val VZ-nál?
ss_parts['Lassúmenet mértéke motorvonatra (km/h)']=ss_parts['Lassúmenet mértéke motorvonatra (km/h)'].fillna(ss_parts['Lassúmenet mértéke (km/h)'])
#todo pontosítás lassúmenet mekkora része az adott szakasznak? - A tervidő segíthet
#todo milyen vonat?? motor? 

In [ ]:
import numpy as np
# igen/nem
ss_parts['Fels.vez. kikapcsolása']=ss_parts['Fels.vez. kikapcsolása'].map({np.nan:0,'X':1})
# ideiglenes az, ami hatással lehet a késésre, elvileg az állandónak már benne kellene lenni a menetrendben 
ss_parts['Lassúmenet típusa']=ss_parts['Lassúmenet típusa'].map({'Á':0,'I':1})
#ss_parts=pd.get_dummies(ss_parts, columns=['Érintett objektum típusa'],dtype=int)


In [ ]:
ss_status = slow_signal_tables['ZVZ_KEIHEAD'].copy()

ss_status.drop(axis=0,index=0,inplace=True)
ss_status['Vágányzár KEI ID']=ss_status['Vágányzár KEI ID'].astype(str).str.replace('.0','')
#ss_status['Utolsó jóváhagyott kezdés dátuma']=pd.to_datetime( ss_status['Utolsó jóváhagyott kezdés dátuma'],format='%Y-%m-%d').dt.date    
#ss_status['Utolsó jóváhagyott kezdés időpontja']=pd.to_datetime( ss_status['Utolsó jóváhagyott kezdés időpontja']).dt.time

#excel típus információ lehet, alapból dátum datetime64[ns] lesz, így visszakovert után formátum változik
ss_status['Kezdés']= pd.to_datetime(ss_status['Utolsó jóváhagyott kezdés dátuma'].astype(str)+' '+ss_status['Utolsó jóváhagyott kezdés időpontja'].astype(str),format='%Y-%m-%d %H:%M:%S')
#ss_status['Utolsó jóváhagyott befejezés dátuma']=pd.to_datetime( ss_status['Utolsó jóváhagyott befejezés dátuma'],format='%Y-%m-%d').dt.date    
#ss_status['Utolsó jóváhagyott befejezés időpontja']=pd.to_datetime( ss_status['Utolsó befejezés kezdés időpontja']).dt.time

#
ss_status['Befejezés']= pd.to_datetime(ss_status['Utolsó jóváhagyott befejezés dátuma'].astype(str)+' '+ss_status['Utolsó jóváhagyott befejezés időpontja'].astype(str),format='%Y-%m-%d %H:%M:%S',errors='coerce')


'''
ss_status['Tény-indítás (dátum)']=pd.to_datetime( ss_status['Tény-indítás (dátum)'],format='%Y-%m-%d').dt.date   
ss_status['TényIndít.időpontja']=pd.to_datetime( ss_status['TényIndít.időpontja']).dt.time
ss_status['Tény-befejez.(dátum)']=pd.to_datetime( ss_status['Tény-befejez.(dátum)'],format='%Y-%m-%d').dt.date   
ss_status['Tény-befej.ideje']=pd.to_datetime( ss_status['Tény-befej.ideje']).dt.time
'''

to_drop_head=['Státusséma','Utolsó jóváhagyott kezdés dátuma','Utolsó jóváhagyott kezdés időpontja','Utolsó jóváhagyott befejezés dátuma','Utolsó jóváhagyott befejezés időpontja','Státusséma','Vágányzár Státus','Korlátozás típus']
ss_status=ss_status.drop(columns=to_drop_head)


In [ ]:
merged=ss_parts.merge(ss_status,how='left',left_on='Kizárásszám',right_on='Vágányzár KEI ID')


In [ ]:
c_head=closure_tables['ZVZ_VZTHEAD'].copy()
# "lemondott" -  akkor ez nem történt meg?
canceled=['KNJM','LEMD','LEMM','LEMO','KELU']
# "törölt" ez mit jelent ? szintén nem történt meg?   
deleted=['ELUT','VPEE','VTÖR','KELU']
not_valid=canceled+deleted
c_head['Státus'] = 1
c_head.loc[c_head['Vágányzár Státus'].isin(not_valid), "Státus"] = 0

c_head['Kiutalt menetvonalak zavartatása nélkül'] = c_head['Kiutalt menetvonalak zavartatása nélkül'].map(
    {np.nan: 0, 'X': 1})
c_head.drop(axis=0,index=0,inplace=True)
c_head['Vágányzár VZT ID']=c_head['Vágányzár VZT ID'].astype(str).str.replace('.0','')
#ss_status['Utolsó jóváhagyott kezdés dátuma']=pd.to_datetime( ss_status['Utolsó jóváhagyott kezdés dátuma'],format='%Y-%m-%d').dt.date    
#ss_status['Utolsó jóváhagyott kezdés időpontja']=pd.to_datetime( ss_status['Utolsó jóváhagyott kezdés időpontja']).dt.time

#excel típus információ lehet, alapból dátum datetime64[ns] lesz, így visszakovert után formátum változik
c_head['Kezdés']= pd.to_datetime(c_head['Tény-indítás (dátum)'].astype(str)+' '+c_head['TényIndít.időpontja'].astype(str),format='%Y-%m-%d %H:%M:%S',errors='coerce')
#ss_status['Utolsó jóváhagyott befejezés dátuma']=pd.to_datetime( ss_status['Utolsó jóváhagyott befejezés dátuma'],format='%Y-%m-%d').dt.date    
#ss_status['Utolsó jóváhagyott befejezés időpontja']=pd.to_datetime( ss_status['Utolsó befejezés kezdés időpontja']).dt.time

#
c_head['Befejezés']= pd.to_datetime(c_head['Tény-befejez.(dátum)'].astype(str)+' '+c_head['Tény-befej.ideje'].astype(str),format='%Y-%m-%d %H:%M:%S',errors='coerce')
#ementrend szempontjából nem érdekes
to_drop_c_head=['Korlátozás típus','Korlátozottan.rend.álló kap.','Státusséma','Vágányzár Státus','Tény-indítás (dátum)','TényIndít.időpontja','Tény-befejez.(dátum)','Tény-befej.ideje','Utolsó jóváhagyott befejezés dátuma','Utolsó jóváhagyott befejezés időpontja','Utolsó jóváhagyott kezdés dátuma','Utolsó jóváhagyott kezdés időpontja','Vágányzár túllépve','Van vonatkésés','Pótolni kell']

c_head.drop(columns=to_drop_c_head,inplace=True)

In [ ]:
c_parts = closure_tables['ZVZ_KIZARASOK'].copy()

c_parts.loc[c_parts['Záró szolgálati hely'].isna(), 'Záró szolgálati hely'] = c_parts['Kezdő szolgálati hely']
c_parts_bw=c_parts.copy()



#TODO  Vágány (VZ) és 'Átmenő vágány típusa' ugyanannak néz ki mint lassújelnél, ugyanazt a logikát csináljuk itt is 
#itt vannak 1,2,3,4 értékek VZ-nél, elvileg lehet 'A' itt is 

c_parts=encode_directions(c_parts,c_parts_bw)
set_section_id(c_parts)

c_parts = c_parts.drop(columns=['Kezdő szolgálati hely','Záró szolgálati hely','Vágány (VZ)','Átmenő vágány típusa']) 
to_drop_gen=['Kizárás tétel','Kizárás típusa','Kezdő SZH statisztikai szám','Záró SZH statisztikai szám','Kis teljesítményű dízel kitoló','Dízel kitoló szükséges','Nagy teljesítményű dízel kitoló', 'Állomási bizt.ber. kikapcsolás szükséges','Vonali bizt. ber. kikapcsolás szükséges',"INF vonal","INF vonal.1","HÜSZ vonal","HÜSZ vonal.1"]

to_drop_ss=['Lassúmenet kezdő szelvény','Lassúmenet záró szelvény','Változó munkahelyen kitűzött','Lassúmenet mértéke tehervonatra (km/h)']
c_parts['Fels.vez. kikapcsolása']=c_parts['Fels.vez. kikapcsolása'].map({np.nan:0,'X':1})

c_parts.drop(columns=to_drop_gen,inplace=True)

#TODO ezen azért lehet majd még gondolkodni...
#c_parts['Lassúmenet mértéke (km/h)']=0.0
#c_parts['Lassúmenet mértéke motorvonatra (km/h)']=0.0
#TODO ez lehet gáz
#c_parts['Lassúmenet hossza']=0
#c_parts['Lassúmenet típusa']=1

c_parts['Kizárásszám']=c_parts['Kizárásszám'].astype(str).str.replace('.0','')

c_merged=c_parts.merge(c_head,how='left',left_on='Kizárásszám',right_on='Vágányzár VZT ID')



In [ ]:
merged['Kizárás (1=VZ,2=LJ)']=2
c_merged['Kizárás (1=VZ,2=LJ)']=1

In [ ]:
line_status=pd.concat([merged,c_merged])
del merged,c_merged
line_status.head()

In [ ]:
final_drop=['Vágányzár KEI ID','Vágányzár VZT ID']
line_status.drop(columns=final_drop,inplace=True)

In [ ]:
line_status.fillna(0.0,inplace=True)


In [ ]:
line_status.head()

In [ ]:
fn='data/line_status.csv'
line_status.to_pickle(fn)
del line_status

# load
df = pd.read_pickle(fn)

In [ ]:
df.head()

# Végső összerakás

In [22]:
def add_line_info(e, line_info):
    #e=events.copy()
    #d2.reset_index(inplace=True)
    joined = e.merge(line_info,on='ID',how='inner')
    # todo miért kell???
    joined['Kezdés']=pd.to_datetime(joined['Kezdés'])
    joined['Befejezés']=pd.to_datetime(joined['Befejezés'])
    #ezek azok az események, ahol volt valami korlátozás
    joined = joined[joined['ELOZO_ESEMENY_IDO'].between(joined['Kezdés'],joined['Befejezés'])]
    e=e.reindex(columns=joined.columns)
    e.update(joined)
    e.fillna(0.0,inplace=True)
    return e



fn='data/line_status.csv'

line_status = pd.read_pickle(fn)
events_with_line_info=add_line_info(d2, line_status)
events_with_line_info.head()
    

C:\Users\ELTE\AppData\Local\Temp\ipykernel_7280\2260000315.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[nan nan nan ... nan nan nan]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  e.update(joined)
C:\Users\ELTE\AppData\Local\Temp\ipykernel_7280\2260000315.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[nan nan nan ... nan nan nan]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  e.update(joined)
C:\Users\ELTE\AppData\Local\Temp\ipykernel_7280\2260000315.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[nan nan nan ... nan nan nan]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  e.update(joined)
C:\Users\ELTE\AppData\Local\Tem

,VONAL,VONATSZAM,VONTATO_TIPUS,ELOZO_ESEMENY_IDO,MENETREND_IDO,OSSZ_KESES,ELOZO_KESES,KESES,RELATIV_KESES,TERV_IDOTARTAM (m),...,Fels.vez. kikapcsolása,I,E,ÁI,ÁE,Kezdés,Befejezés,"Kizárás (1=VZ,2=LJ)",Kiutalt menetvonalak zavartatása nélkül,Státus
0,70,2010,465,0.0,00:25:00,0 days 00:00:24,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,70,2010,465,2022-01-01 00:25:00,00:29:00,0 days 00:01:10,0 days 00:00:24,0 days 00:00:46,0.191667,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2010,465,2022-01-01 00:29:00,00:30:00,0 days 00:00:40,0 days 00:01:10,-1 days +23:59:30,-0.500000,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2,2010,465,2022-01-01 00:30:00,00:35:00,-1 days +23:59:55,0 days 00:00:40,-1 days +23:59:15,-0.150000,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2,2010,465,2022-01-01 00:35:00,00:36:00,0 days 00:00:09,-1 days +23:59:55,0 days 00:00:14,0.233333,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# https://stackoverflow.com/questions/50422404/save-and-export-dtypes-information-of-a-python-pandas-dataframe
def save_data_with_types(df,fn):
    with open(fn, 'wt') as f:
        f.write(',' + ','.join(map(str, df.dtypes)) + '\n')
        df.to_csv(f, lineterminator='\n')

def load_data(fn): 
    with open(fn, 'rt') as f:
        types = next(f).rstrip().split(',')[1:]
        columns = next(f).rstrip().split(',')[1:]
        df = pd.read_csv(f, dtype=dict(zip(columns, types)), index_col=0, names=columns)
        return df

In [23]:
def add_previous_train_delay(e):
    #e=events.copy()
    #TODO érdemes lehet mozdonyid-ra is csoportosítani
    e[['ELOZO VONAT KESES','ELOZO STATUS','ELOZO VONAT RELATIV KESES']]=e.groupby('ID')[['KESES','Kizárásszám','RELATIV_KESES']].shift()
    e['VONAL STATUSZ VALTOZOTT']=e['ELOZO STATUS']!=e['Kizárásszám']
    e.drop(columns=['ELOZO STATUS','ELOZO STATUS'],inplace=True)
    return e

events_with_line_info=add_previous_train_delay(events_with_line_info)

In [ ]:
events_with_line_info.loc[events_with_line_info['ID']=="10033-10082"]

In [24]:
del events_with_line_info

In [25]:
events=[] 
for year in years_to_process:
    for line in [2,147]:
        #esemény és időjárás adat
        e = get_events_pred_data(get_events_data(weather_df,coord_dict,year,line),line)
        e = add_line_info(e,line_status)
        e = add_previous_train_delay(e)
        events.append(e)
data=pd.concat(events)

C:\Users\ELTE\AppData\Local\Temp\ipykernel_7280\2781791593.py:56: DtypeWarning: Columns (16,17,28) have mixed types. Specify dtype option on import or set low_memory=False.
  e_=pd.read_csv(l+event_file_name+str(line)+'.txt',sep=',',encoding='iso-8859-2')


2022-01-01 00:00:00  -  2023-12-31 00:00:00


C:\Users\ELTE\AppData\Local\Temp\ipykernel_7280\2260000315.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[nan nan nan ... nan nan nan]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  e.update(joined)
C:\Users\ELTE\AppData\Local\Temp\ipykernel_7280\2260000315.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[nan nan nan ... nan nan nan]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  e.update(joined)
C:\Users\ELTE\AppData\Local\Temp\ipykernel_7280\2260000315.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[nan nan nan ... nan nan nan]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  e.update(joined)
C:\Users\ELTE\AppData\Local\Tem

2022-01-01 00:00:00  -  2023-12-31 00:00:00


C:\Users\ELTE\AppData\Local\Temp\ipykernel_7280\2260000315.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[nan nan nan ... nan nan nan]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  e.update(joined)
C:\Users\ELTE\AppData\Local\Temp\ipykernel_7280\2260000315.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[nan nan nan ... nan nan nan]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  e.update(joined)
C:\Users\ELTE\AppData\Local\Temp\ipykernel_7280\2260000315.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[nan nan nan ... nan nan nan]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  e.update(joined)
C:\Users\ELTE\AppData\Local\Tem

2023-01-01 00:00:00  -  2023-12-31 00:00:00


C:\Users\ELTE\AppData\Local\Temp\ipykernel_7280\2260000315.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[nan nan nan ... nan nan nan]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  e.update(joined)
C:\Users\ELTE\AppData\Local\Temp\ipykernel_7280\2260000315.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[nan nan nan ... nan nan nan]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  e.update(joined)
C:\Users\ELTE\AppData\Local\Temp\ipykernel_7280\2260000315.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[nan nan nan ... nan nan nan]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  e.update(joined)
C:\Users\ELTE\AppData\Local\Tem

2023-01-01 00:00:00  -  2023-12-31 00:00:00


C:\Users\ELTE\AppData\Local\Temp\ipykernel_7280\2260000315.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[nan nan nan ... nan nan nan]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  e.update(joined)
C:\Users\ELTE\AppData\Local\Temp\ipykernel_7280\2260000315.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[nan nan nan ... nan nan nan]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  e.update(joined)
C:\Users\ELTE\AppData\Local\Temp\ipykernel_7280\2260000315.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[nan nan nan ... nan nan nan]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  e.update(joined)
C:\Users\ELTE\AppData\Local\Tem

In [ ]:
# TODO nem az igazi, pkl helyette most
#save_data_with_types(data,data_root+'data.csv')

    

In [26]:
data.to_pickle(data_root+'data.pkl')

In [ ]:

print("hello")
